# **TP3**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

## **Load data**

In [2]:
data_path = '..//DATA//DatasetExos.csv'

In [3]:
def load_data(path):
    try:
        data = pd.read_csv(path , sep=',')
        return data
    except Exception as e:
      print(f"Une erreur s'est produite : {e}")

In [4]:
data = load_data(data_path)

In [5]:
data.head(10)

,ep (ms),Acc_x,Acc_y,Acc_z,Gyro_x,Gyro_y,Gyro_z,ID,Exercise,Category,Set
0,2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,30.0
1,2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,30.0
2,2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,30.0
3,2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,30.0
4,2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,30.0
5,2019-01-11 15:08:06.200,-0.026000,0.965000,-0.118000,0.4634,5.2194,-6.4636,B,bench,heavy,30.0
6,2019-01-11 15:08:06.400,-0.048667,0.790000,-0.145333,21.6950,8.1708,-28.2196,B,bench,heavy,30.0
7,2019-01-11 15:08:06.600,-0.170000,0.899500,-0.250000,17.5246,1.5976,-17.5854,B,bench,heavy,30.0
8,2019-01-11 15:08:06.800,-0.222667,0.907000,-0.204333,-7.2318,-1.3536,-0.4026,B,bench,heavy,30.0
9,2019-01-11 15:08:07.000,-0.204500,0.930000,-0.149000,-28.6830,-10.2076,20.5732,B,bench,heavy,30.0


## **Descitisation**

### **Methode1**

In [6]:
def Position_Quantile(Q,N):
    Quantiles = {}
    for j in range(1,Q):
        Quantiles[f'Q{j}']=int((N*j)/Q)
    return Quantiles

In [7]:
def Discretization_Equal_Frequency_Intervals(data, Q=1):
    intervalle = {}
    N = len(data)
    # Trier les données pour garantir que les intervalles soient ordonnés
    data_sorted = sorted(data)
    # Calcul des quantiles
    Quantiles = Position_Quantile(Q, N)
    # Ajouter un intervalle final pour couvrir toutes les données
    Quantiles[f'Q{Q}'] = N
    print(Quantiles)

    # Diviser les données en intervalles selon les quantiles
    previous_index = 0
    for q, index in Quantiles.items():
        print(f" q:{q} and index :{index}")
        intervalle[q] = data_sorted[previous_index:index]
        previous_index = index
    
    return intervalle


In [12]:
print(f"Nombre de données : {len(data['Acc_x'])}")

Q = 4  # Nombre de quantiles
intervalles = Discretization_Equal_Frequency_Intervals(data['Acc_x'], Q)

for nom_intervalle, valeurs in intervalles.items():
    taille_intervalle = len(valeurs)  # Taille de l'intervalle
    print(f"Intervalle : {nom_intervalle},  Taille : {taille_intervalle} , Valeurs : {valeurs}")

Nombre de données : 9009
{'Q1': 2252, 'Q2': 4504, 'Q3': 6756, 'Q4': 9009}
 q:Q1 and index :2252
 q:Q2 and index :4504
 q:Q3 and index :6756
 q:Q4 and index :9009
Intervalle : Q1,  Taille : 2252 , Valeurs : [-0.518, -0.513333333333333, -0.491, -0.4895, -0.489, -0.4795, -0.476666666666667, -0.4745, -0.472, -0.469333333333333, -0.455, -0.455, -0.451, -0.45, -0.45, -0.448, -0.445666666666667, -0.442333333333333, -0.441, -0.44, -0.44, -0.439, -0.4385, -0.438333333333333, -0.437333333333333, -0.435666666666667, -0.435666666666667, -0.434, -0.433666666666667, -0.4325, -0.432, -0.432, -0.431666666666667, -0.4315, -0.430666666666667, -0.4305, -0.43, -0.4285, -0.428, -0.428, -0.4245, -0.423, -0.4225, -0.421, -0.420666666666667, -0.416, -0.415333333333333, -0.415, -0.4115, -0.4105, -0.4105, -0.408333333333333, -0.407333333333333, -0.406333333333333, -0.406, -0.406, -0.4055, -0.405, -0.4035, -0.403333333333333, -0.402333333333333, -0.401, -0.401, -0.400666666666667, -0.400333333333333, -0.3995, -0

### **Methode2**

In [8]:
def Discretization_Equal_Width_Intervals(data, W=1):
    intervalles = {}
    N = len(data)
    
    # Trier les données
    data_sorted = sorted(data)
    
    max_data = max(data_sorted)
    min_data = min(data_sorted)
    print(f"min : {min_data}  max : {max_data}")
    
    # Calculer la largeur de l'intervalle
    largeur_intervalle = (max_data - min_data) / W
    print(f"Largeur de l'intervalle : {largeur_intervalle}")
    
    index = 0
    for i in range(W):  # Commencer à 0 pour inclure le premier intervalle
        
        # # Calculer la borne supérieure de l'intervalle
        borne_superieure = min(min_data + (i + 1) * largeur_intervalle, max_data)
        born_inferieur = min_data + i * largeur_intervalle
        print(f"born sup : {borne_superieure} born inf : {born_inferieur}")
        # Ajouter l'intervalle
        intervalles[f'W{i + 1}'] = [val for val in data_sorted if  born_inferieur<= val < borne_superieure]
        
    return intervalles



a cause des valeurs aberantes de acc_x les autre intervalle sont vide

In [19]:
intervalles = Discretization_Equal_Width_Intervals(data['Acc_x'], W=4)

for nom_intervalle, valeurs in intervalles.items():
    taille_intervalle = len(valeurs)  # Taille de l'intervalle
    print(f"Intervalle : {nom_intervalle},  Taille : {taille_intervalle} , Valeurs : {valeurs}")

min : -9907.0225  max : 100000.07
Largeur de l'intervalle : 27476.773125
born sup : 17569.750625 born inf : -9907.0225
born sup : 45046.52375 born inf : 17569.750625
born sup : 72523.296875 born inf : 45046.52375
born sup : 100000.07 born inf : 72523.296875
Intervalle : W1,  Taille : 9004 , Valeurs : [-0.518, -0.513333333333333, -0.491, -0.4895, -0.489, -0.4795, -0.476666666666667, -0.4745, -0.472, -0.469333333333333, -0.455, -0.455, -0.451, -0.45, -0.45, -0.448, -0.445666666666667, -0.442333333333333, -0.441, -0.44, -0.44, -0.439, -0.4385, -0.438333333333333, -0.437333333333333, -0.435666666666667, -0.435666666666667, -0.434, -0.433666666666667, -0.4325, -0.432, -0.432, -0.431666666666667, -0.4315, -0.430666666666667, -0.4305, -0.43, -0.4285, -0.428, -0.428, -0.4245, -0.423, -0.4225, -0.421, -0.420666666666667, -0.416, -0.415333333333333, -0.415, -0.4115, -0.4105, -0.4105, -0.408333333333333, -0.407333333333333, -0.406333333333333, -0.406, -0.406, -0.4055, -0.405, -0.4035, -0.40333333

suppression des valeurs aberantes

In [9]:
filtred_data = data[(data['Acc_x'] <= 2000) & (data['Acc_x'] >= -1000)]

In [ ]:
filtred_data = data[(data['Acc_x'] <= 2000) & (data['Acc_x'] >= -1000)]

intervalles = Discretization_Equal_Width_Intervals(filtred_data['Acc_x'], W=4)

for nom_intervalle, valeurs in intervalles.items():
    taille_intervalle = len(valeurs)  # Taille de l'intervalle
    print(f"Intervalle : {nom_intervalle},  Taille : {taille_intervalle} , Valeurs : {valeurs}")

min : -0.838  max : 1.464
Largeur de l'intervalle : 0.5755
born sup : -0.26249999999999996 born inf : -0.838
born sup : 0.31300000000000006 born inf : -0.26249999999999996
born sup : 0.8885000000000002 born inf : 0.31300000000000006
born sup : 1.464 born inf : 0.8885000000000002
Intervalle : W1,  Taille : 923 , Valeurs : [-0.838, -0.838, -0.838, -0.7755, -0.7755, -0.7755, -0.7745, -0.7745, -0.7745, -0.773333333333333, -0.773333333333333, -0.773333333333333, -0.745333333333333, -0.745333333333333, -0.745333333333333, -0.710333333333333, -0.710333333333333, -0.710333333333333, -0.703, -0.703, -0.703, -0.681, -0.681, -0.681, -0.543, -0.518, -0.513333333333333, -0.491, -0.4895, -0.489, -0.4815, -0.48, -0.4795, -0.476666666666667, -0.4745, -0.472, -0.4715, -0.4715, -0.4715, -0.469333333333333, -0.455, -0.455, -0.451, -0.45, -0.45, -0.448, -0.447, -0.445666666666667, -0.445, -0.443333333333333, -0.442333333333333, -0.441, -0.44, -0.44, -0.439, -0.4385, -0.438333333333333, -0.437333333333333,

## **Normalization**

### **Z-score Normalization**

In [30]:
def Z_score(data):
    mean = np.mean(data)
    std = np.std(data)
    z = (data - mean) / std
    return z

In [31]:
data_normalized = Z_score(data['Acc_x'])
print(data_normalized)

0      -0.009472
1      -0.009486
2      -0.009484
3      -0.009508
4      -0.009511
          ...   
9004   -0.009530
9005   -0.009520
9006   -0.009542
9007   -0.009521
9008   -0.009527
Name: Acc_x, Length: 9009, dtype: float64


### **Min-Max Normalization**

In [10]:
def Min_Max_Scaling(data_col , new_min = 0, new_max = 1):
    return ((data_col- min(data_col)) / (max(data_col) - min(data_col)))*(new_max - new_min) + new_min

In [13]:
data_normalized = Min_Max_Scaling(data['Acc_x'])
print(data_normalized)

0       0.090140
1       0.090140
2       0.090140
3       0.090140
4       0.090140
          ...   
9004    0.090140
9005    0.090140
9006    0.090139
9007    0.090140
9008    0.090140
Name: Acc_x, Length: 9009, dtype: float64


## **Visualisation**

In [11]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

filtred_data = data[(data['Acc_x'] <= 2000) & (data['Acc_x'] >= -1000)]

# Appliquer les méthodes de prétraitement et stocker les résultats
discret_freq = Discretization_Equal_Frequency_Intervals(filtred_data['Acc_x'], Q=4)
discret_width = Discretization_Equal_Width_Intervals(filtred_data['Acc_x'], W=4)
normalized_min_max = Min_Max_Scaling(data_col=filtred_data['Acc_x'])



{'Q1': 2250, 'Q2': 4501, 'Q3': 6752, 'Q4': 9003}
 q:Q1 and index :2250
 q:Q2 and index :4501
 q:Q3 and index :6752
 q:Q4 and index :9003
min : -0.838  max : 1.464
Largeur de l'intervalle : 0.5755
born sup : -0.26249999999999996 born inf : -0.838
born sup : 0.31300000000000006 born inf : -0.26249999999999996
born sup : 0.8885000000000002 born inf : 0.31300000000000006
born sup : 1.464 born inf : 0.8885000000000002


In [ ]:
# Histogramme avant et après pour chaque méthode
fig, axs = plt.subplots(4, 2, figsize=(12, 16))

# Avant les transformations
sns.histplot(data, kde=True, ax=axs[0, 0]).set(title='Avant Discrétisation (Fréquence Égale)')
sns.histplot(data, kde=True, ax=axs[1, 0]).set(title='Avant Discrétisation (Largeur Égale)')
sns.histplot(data, kde=True, ax=axs[2, 0]).set(title='Avant Normalisation Min-Max')


# Après transformations
sns.histplot([item for sublist in discret_freq.values() for item in sublist], kde=True, ax=axs[0, 1]).set(title='Après Discrétisation (Fréquence Égale)')
sns.histplot([item for sublist in discret_width.values() for item in sublist], kde=True, ax=axs[1, 1]).set(title='Après Discrétisation (Largeur Égale)')
sns.histplot(normalized_min_max, kde=True, ax=axs[2, 1]).set(title='Après Normalisation Min-Max')


plt.tight_layout()
plt.show()